In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
def order_points(pts):
    '''Rearrange coordinates to order:
      top-left, top-right, bottom-right, bottom-left'''
    rect = np.zeros((4, 2), dtype='float32')
    pts = np.array(pts)
    s = pts.sum(axis=1)
    # Top-left point will have the smallest sum.
    rect[0] = pts[np.argmin(s)]
    # Bottom-right point will have the largest sum.
    rect[2] = pts[np.argmax(s)]
 
    diff = np.diff(pts, axis=1)
    # Top-right point will have the smallest difference.
    rect[1] = pts[np.argmin(diff)]
    # Bottom-left will have the largest difference.
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect.astype('int').tolist()

def find_dest(pts):
    (tl, tr, br, bl) = pts
    # Finding the maximum width.
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
 
    # Finding the maximum height.
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # Final destination co-ordinates.
    destination_corners = [[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]
 
    return order_points(destination_corners)

'''
def pad(corners, padding):
	x1, y1 = corners[0]
	x2, y2 = corners[1]
	x3, y3 = corners[2]
	x4, y4 = corners[3]

	xc, yc = ((x1+x2+x3+x4)/4, (y1+y2+y3+y4)/4) # centroid

	cp1 = ((x1-xc)*padding, (y1-yc)*padding)
	cp2 = ((x2-xc)*padding, (y2-yc)*padding)
	cp3 = ((x3-xc)*padding, (y3-yc)*padding)
	cp4 = ((x4-xc)*padding, (y4-yc)*padding)

	x1 = xc + cp1[0]
	y1 = yc + cp1[1]
	x2 = xc + cp2[0]
	y2 = yc + cp2[1]
	x3 = xc + cp3[0]
	y3 = yc + cp3[1]
	x4 = xc + cp4[0]
	y4 = yc + cp4[1]

	new_corners = [[x1,y1], [x2, y2], [x3,y3], [x4,y4]]

	return new_corners
'''
# expands corners horizontally to account for the small width between ArUco markers and the scoresheet's edges
def pad(corners, padding):
	x1, y1 = corners[0]
	x2, y2 = corners[1]
	x3, y3 = corners[2]
	x4, y4 = corners[3]

	dx_12 = (x2 - x1) * padding
	dy_12 = (y2 - y1) * padding

	dx_43 = (x3 - x4) * padding
	dy_43 = (y3 - y4) * padding

	x1 -= dx_12
	y1 -= dy_12

	x2 += dx_12
	y2 += dy_12

	x3 += dx_43
	y3 += dy_43

	x4 -= dx_43
	y4 -= dy_43

	new_corners = [[x1,y1], [x2, y2], [x3,y3], [x4,y4]]

	return new_corners


def scan(img):

	# Create a copy of resized original image for later use
	resized_img = img.copy()

	# Resize image to workable size
	dim_limit = 1080
	max_dim = max(img.shape)
	if max_dim > dim_limit:
		resize_scale = dim_limit / max_dim
		resized_img = cv2.resize(img, None, fx=resize_scale, fy=resize_scale)

	dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
	parameters =  cv2.aruco.DetectorParameters()
	detector = cv2.aruco.ArucoDetector(dictionary, parameters)

	corners, ids, _ = detector.detectMarkers(resized_img)

	img_corners = []

	# verify that *exactly* 4 ArUCo markers were found
	if [[i] in ids for i in range(4)]:
		# flatten the ArUco IDs list
		ids = ids.flatten()
		# loop over the detected ArUCo corners
		for (markerCorner, markerID) in zip(corners, ids):
			# extract the marker corners (which are always returned in
			# top-left, top-right, bottom-right, and bottom-left order)
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners

			if markerID == 0:
				img_corners.append(topLeft)
			if markerID == 1:
				img_corners.append(topRight)
			if markerID == 2:
				img_corners.append(bottomLeft)
			if markerID == 3:
				img_corners.append(bottomRight)
	else:
		raise Exception("Incorrect number of ArUco markers detected.")

	# For 4 corner points being detected.
	img_corners = order_points(img_corners)

	# Horizontally pad the corners
	img_corners = pad(img_corners, 1/65)

	img_corners = [[corner[0]/resize_scale, corner[1]/resize_scale] for corner in img_corners]
	destination_corners = find_dest(img_corners)


	h, w = img.shape[:2]
	# Getting the homography.
	M = cv2.getPerspectiveTransform(np.float32(img_corners), np.float32(destination_corners))
	# Perspective transform using homography.
	final = cv2.warpPerspective(img, M, (destination_corners[2][0], destination_corners[2][1]),
								flags=cv2.INTER_LINEAR)

	return final
			

img = cv2.imread("example.png", cv2.IMREAD_COLOR)
aligned_img = scan(img)

def resize(img, new_w, new_h):
	h, w, _ = img.shape
	w_resize_scale = new_w / w
	h_resize_scale = new_h / h

	resized_img = cv2.resize(img, None, fx=w_resize_scale, fy=h_resize_scale)
	return resized_img

# standardize each image's size, since later on in pre-processing we use specific pixel sizes to remove noise, fill gaps, etc.
aligned_img = resize(aligned_img, 2048, 3736)
cv2.imwrite("aligned.png", aligned_img)


True

In [2]:
int(cv2.__version__.split(".")[1])

10